In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv('../data/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
Citation : "https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5"
from nltk.corpus import stopwords
import re
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    
    return text

df['lowered_comment_text'] = df['comment_text'].map(lambda text : clean_text(text.lower()))
df['comment_text'] = df['comment_text'].map(lambda text : clean_text(text))
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,lowered_comment_text
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0,explanation why the edits made under my userna...
1,000103f0d9cfb60f,D aww He matches this background colour I m se...,0,0,0,0,0,0,d aww he matches this background colour i am s...
2,000113f07ec002fd,Hey man I m really not trying to edit war It j...,0,0,0,0,0,0,hey man i am really not trying to edit war it ...
3,0001b41b1c6bb37e,More I can not make any real suggestions on i...,0,0,0,0,0,0,more i can not make any real suggestions on i...
4,0001d958c54c6e35,You sir are my hero Any chance you remember wh...,0,0,0,0,0,0,you sir are my hero any chance you remember wh...
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,And for the second time of asking when your v...,0,0,0,0,0,0,and for the second time of asking when your v...
159567,ffea4adeee384e90,You should be ashamed of yourself That is a ho...,0,0,0,0,0,0,you should be ashamed of yourself that is a ho...
159568,ffee36eab5c267c9,Spitzer Umm theres no actual article for prost...,0,0,0,0,0,0,spitzer umm theres no actual article for prost...
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,and it looks like it was actually you who put ...


In [13]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train, test = train_test_split(df,test_size=0.15,random_state = 42)
Xtrain = {"UnLoweredText": train.comment_text,"LoweredText":train.lowered_comment_text}
X_test = test.comment_text

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

# Build Pipelines
# tdiff - countvect


stop_words = set(stopwords.words('english'))
# Define a pipeline combining a text feature extractor with multi lablel classifier

# Create multiple models

models = {}

def createModels(ngrams):

    model = "NBmodelTfdiff"

    models[model] = Pipeline([
                    ('tfidf', TfidfVectorizer(stop_words=stop_words,min_df=5,sublinear_tf=True)),
                    ('clf', OneVsRestClassifier(MultinomialNB(
                        fit_prior=True, class_prior=None))),
                ])

    model = "RFmodelTfdiff"
    models[model] = Pipeline([
                    ('tfidf', TfidfVectorizer(stop_words=stop_words,min_df=5,sublinear_tf=True)),
                    ('clf', OneVsRestClassifier(RandomForestClassifier(random_state = 1,n_estimators = 10,))),
                ])
    model = "LRmodelTfdiff"
    models[model] = Pipeline([
                        ('tfidf', TfidfVectorizer(stop_words=stop_words,min_df=5,sublinear_tf=True)),
                        ('clf', OneVsRestClassifier(LogisticRegression(random_state = 1))),
                    ])
    model = "NBmodelCountdiff"
    models[model] = Pipeline([
                    ('vect',CountVectorizer(min_df=5)),
                    ('clf', OneVsRestClassifier(MultinomialNB(
                        fit_prior=True, class_prior=None)))])
    model = "RFmodelCountdiff"
    models[model] = Pipeline([
                    ('vect', CountVectorizer(min_df=5)),
                    ('clf', OneVsRestClassifier(RandomForestClassifier(random_state = 1,n_estimators = 10,))),
                ])
    model = "LRmodelCountdiff"
    models[model] = Pipeline([
                    ('vect', CountVectorizer(min_df=5)),
                    ('clf', OneVsRestClassifier(LogisticRegression(random_state = 1))),
                ])
    return models
    

ngrams = {"unigram":(1,1),"bigram":(1,2),"trigrams":(1,3)}
models = createModels(ngrams)
print(models)

             

{'NBmodelTfdiff': Pipeline(steps=[('tfidf',
                 TfidfVectorizer(min_df=5,
                                 stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...},
                                 sublinear_tf=True)),
                ('clf', OneVsRestClassifier(estimator=MultinomialNB()))]), 'RFmodelTfdiff': Pipeline(steps=[('tfidf',
                 TfidfVectorizer(min_df=5,
                                 stop_words={'a', 'about', 'above', 'after',
            

In [27]:
pred_df = pd.DataFrame()

def trainModel(dataframe):
    for trainset in Xtrain:
        for model in models:
            scores = []
            pipeline = models[model]
            row = model + trainset
            for category in categories:
                print('... Processing {}'.format(category))
                # train the model using X_dtm & y
                pipeline.fit(Xtrain[trainset], train[category])
                # compute the testing accuracy
                prediction = pipeline.predict(X_test)
                accuracy = accuracy_score(test[category], prediction)
                scores.append(accuracy)
            df2 = {'Model':row,'Accuracy': sum(scores) / len(scores)} 
            dataframe = dataframe.append(df2, ignore_index = True) 
            scores = []
    return dataframe

pred_df = trainModel(pred_df)
pred_df  

... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic


/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic
... Processing severe_toxic
... Processing obscene
... Processing threat
... Processing insult
... Processing identity_hate
... Processing toxic


/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


... Processing severe_toxic


...

... Processing insult


/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


... Processing identity_hate


/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Accuracy,Model
0,0.974975,NBmodelTfdiffUnLoweredText
1,0.980051,RFmodelTfdiffUnLoweredText
2,0.981339,LRmodelTfdiffUnLoweredText
3,0.967740,NBmodelCountdiffUnLoweredText
4,0.976284,RFmodelCountdiffUnLoweredText
5,0.980274,LRmodelCountdiffUnLoweredText
6,0.974989,NBmodelTfdiffLoweredText
7,0.979334,RFmodelTfdiffLoweredText
8,0.981346,LRmodelTfdiffLoweredText
9,0.967726,NBmodelCountdiffLoweredText


In [28]:
sorteddf = pred_df['Accuracy'].argmax()
pred_df.iloc[sorteddf]

Accuracy                    0.981346
Model       LRmodelTfdiffLoweredText
Name: 8, dtype: object

In [29]:
maxrow = pred_df['Accuracy'].argmax() 
pred_df.iloc[maxrow]

Accuracy                    0.981346
Model       LRmodelTfdiffLoweredText
Name: 8, dtype: object

In [30]:
import pandas as pd
testdf = pd.read_csv('../data/test.csv')
testlabelsdf = pd.read_csv('../data/test_labels.csv')
testdata = pd.merge(testdf, testlabelsdf, on='id', how='outer')

testdata.drop(testdata[testdata['toxic'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['severe_toxic'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['obscene'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['threat'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['insult'] == -1].index, inplace = True)
testdata.drop(testdata[testdata['identity_hate'] == -1].index, inplace = True)
testdata

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [31]:
categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# train, test = train_test_split(df,test_size=0.15,random_state = 42)


pred_df = pd.DataFrame()

def predict(model):
    modelAccuracy = []
    for category in categories:
        model.fit(Xtrain['LoweredText'], train[category])
        prediction =  model.predict(testdata['comment_text'])
        accuracy = accuracy_score(testdata[category], prediction)
        modelAccuracy.append(accuracy)
        print('Category '+ category)
        print(accuracy)
    print('Accuracy on the whol Test Data:')
    print(sum(modelAccuracy) / len(modelAccuracy))
    

predict(models['LRmodelTfdiff'])
# predict(fittedmodels['LRmodelTfdiffUnLoweredTextUnLoweredText'])
# print(models)


/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Category toxic
0.937712963831317
Category severe_toxic
0.9932476788896183
Category obscene
0.9670042827221857
Category threat
0.9966863609365719
Category insult
0.9642377067116822
Category identity_hate
0.9901372346744194
Accuracy on the whol Test Data:
0.9748377046276323


In [50]:
from tqdm import tqdm

categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

results = {}

def predict(model):
    global results
    modelAccuracy = []
    results[model] = {}
    for category in tqdm(categories):
        topmodel = models[model]
        topmodel.fit(Xtrain['LoweredText'], train[category])
        prediction =  topmodel.predict_proba(testdata['comment_text'])
        results[model][category] = prediction
        #accuracy = accuracy_score(testdata[category], prediction)
        #modelAccuracy.append(accuracy)
        #print('Category '+ category)
        #print(accuracy)
    #print('Accuracy on the whol Test Data:')
    #print(sum(modelAccuracy) / len(modelAccuracy))
predict('RFmodelCountdiff')

100%|██████████| 6/6 [03:04<00:00, 30.69s/it]


In [51]:
import pickle as pkl
pkl.dump(results, open('./results_nb.pkl', 'wb'))

In [52]:
from sklearn.metrics import roc_curve, auc
for model_name, model_results in results.items():
    print(model_name)
    auc_score_lst = []
    for category, y_pred_label in model_results.items():
        y_pred_label = y_pred_label[:,1]
        y_label = testdata[category]
        fpr, tpr, _ = roc_curve(y_label, y_pred_label)
        auc_score = auc(fpr, tpr)
        print('\t{}: {}'.format(category, auc_score))
        auc_score_lst.append(auc_score)

RFmodelCountdiff
	toxic: 0.9195881807551368
	severe_toxic: 0.9066093653279254
	obscene: 0.9361160676221201
	threat: 0.8583203572068543
	insult: 0.9202934661733997
	identity_hate: 0.8636906412092052


In [53]:
Citation : "https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5"

In [54]:
import numpy as np
np.mean(auc_score_lst)

0.9007696797157735

In [33]:
import pandas as pd
from collections import Counter




# Helper Functions:
# function to clean up excessive spacing and special characters / facial expressions

# function to remove accents from text
def remove_accents(a):
    return unidecode.unidecode(a.decode('utf-8'))

# function to remove emojis from text
def deEmojify(inputString):
    inputString = str(inputString)
    return inputString.encode('ascii', 'ignore').decode('ascii')
def clean_up(a):
    a = a.replace("<br />", " ")
    a = a.replace("\[?\[.+?\]?\]", " ")
    a = a.replace("\/{3,}", " ")
    a = a.replace("\&\#.+\&\#\d+?;", " ")
    a = a.replace("• ", "")
    a = a.replace("\d+\&\#\d+?;", " ")
    a = a.replace("\&\#\d+?;", " ")

    # handling facial expressions
    a = a.replace("\:\|", "")
    a = a.replace("\:\)", "")
    a = a.replace("\:\(", "")
    a = a.replace("\:\/", "")

    # replacing excessive spacing
    a = a.replace("\n", " ")
    a = a.replace("\s{2,}", " ")

    return a

# function that applies textblob sentiment analysis to a sentence
def textblob_sent(sentence):
    temp = TextBlob(sentence).sentiment[0]
    return round(temp, 3)

def cleanTweet(text):
    text=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    return text
#     Citation:https://stackoverflow.com/questions/8376691/how-to-remove-hashtag-user-link-of-a-tweet-using-regular-expression

def compute_sentiment(csvdata):
    df = pd.read_csv(csvdata)
    df_final = pd.read_csv(csvdata, index_col=0)
    df = df[["text"]]
    # apply the cleaning methods above
    print("\nCleaning up tweet content...")
    df['text'] = df['text'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    df["text"] = clean_up(df["text"])
    df['text'] = df["text"].str.lower()
    df['text'] = df['text'].replace('\n', ' ', regex=True)
    df['text'] = df['text'].apply(deEmojify)
    df['text'] = df['text'].map(lambda text : cleanTweet(text))
    return df
    print("Cleaned.")

In [34]:
AntiAsiantweets = compute_sentiment('../Tweet-Scraping/Anti-Asian_tweets.csv')
AntiAsiantweets

/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (0,3,6,7,8,10,15,16,17,22,27,28,29,30,31,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):



Cleaning up tweet content...


,text
0,your president was calling it the china virus
1,i think it is fentanyl from wuhan china that i...
2,i m disgusted too this violence must stop but ...
3,the 1 7t taxpayer paided for spending on every...
4,chinx blog ind errr we played defensively and ...
...,...
28405,so at what point are you going to do the right...
28406,in 2011 12 french montana put me on lil durk a...
28407,there s a lot of not wuhan between wuhan and uk
28408,me hear seh a 18k fi it me rather dead with a ...


In [35]:
def predictTweets(model,df):
    modelAccuracy = []
    idx = 1
    for category in categories:
        model.fit(Xtrain['LoweredText'], train[category])
        prediction =  model.predict(df['text'])
        df.insert(idx, category, prediction, allow_duplicates = True)
        idx = idx+1
        print("Done")
    return df
    
    

AntiAsiantweets = predictTweets(models['LRmodelTfdiff'],AntiAsiantweets)

/Users/louismurerwa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Done
Done
Done
Done
Done
Done


In [36]:
AntiAsiantweets.to_csv('AntiAsianTweetsPredictions.csv')